# Download the saved models from pth files

In [ ]:
import gdown
!gdown 1wFB32vUF3ZuesxYZdfEyVtk-nvpYgKFO

Downloading...
From: https://drive.google.com/uc?id=1wFB32vUF3ZuesxYZdfEyVtk-nvpYgKFO
To: /content/models.zip
100% 5.10G/5.10G [01:46<00:00, 48.1MB/s]


In [ ]:
!unzip ./models.zip -d ./models


  inflating: ./models/models/8_datasets/blur_jpeg_5/resnet18/checkpoint.pth  
  inflating: ./models/models/8_datasets/blur_jpeg_5/vgg11/checkpoint.pth  
  inflating: ./models/models/2_datasets/no_aug/resnet50/checkpoint.pth  
  inflating: ./models/models/2_datasets/blur_jpeg_5/resnet50/checkpoint.pth  
   creating: ./models/models/new_dataset/progan2/None_datasets/gaussian/resnet50/
   creating: ./models/models/new_dataset/progan2/None_datasets/blur_jpeg_1/resnet50/
   creating: ./models/models/new_dataset/progan2/None_datasets/no_aug/resnet50/
   creating: ./models/models/new_dataset/progan2/None_datasets/jpeg/resnet50/
   creating: ./models/models/new_dataset/progan2/None_datasets/blur_jpeg_5/resnet50/
   creating: ./models/models/new_dataset/progan2/20_classes/gaussian/resnet50/
   creating: ./models/models/new_dataset/progan2/20_classes/blur_jpeg_1/resnet50/
   creating: ./models/models/new_dataset/progan2/20_classes/no_aug/resnet50/
   creating: ./models/models/new_dataset/progan

In [ ]:
!git clone https://github.com/Joool/ReproducabilityCNNEasyToSpot.git

Cloning into 'ReproducabilityCNNEasyToSpot'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 247 (delta 10), reused 225 (delta 0), pack-reused 0
Receiving objects: 100% (247/247), 2.24 MiB | 5.21 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
!cp -r ./ReproducabilityCNNEasyToSpot/gandetect ./
import gandetect
!pip install torch_dct

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import gandetect
# Load the saved model from file
all_data_jpeg = torch.load('/content/models/models/20_datasets/jpeg/resnet50/checkpoint.pth')
# all_data_no_aug = torch.load()
# all_data_gaussian = torch.load()
# all_data_blur_jpeg_1 = 
# all_data_blur_jpeg_5 = 


In [ ]:
all_data_jpeg

In [ ]:
import argparse
import multiprocessing as mp
from pathlib import Path

import numpy as np
import torch
from PIL import Image
from sklearn.metrics import average_precision_score
from tqdm import tqdm

from gandetect.dataloader import load_data
from gandetect.transforms import TEST_TRANSFORM, TEST_TRANSFORM_DCT
from gandetect.utils import set_seed

EVAL_DATASETS = {
    "ProGAN": "progan",
    "StyleGAN": "stylegan",
    "StyleGAN2": "stylegan2",
    "BigGAN": "biggan",
    "CycleGAN": "cyclegan",
    "StarGAN": "stargan",
    "GauGAN": "gaugan",
    "CRN": "crn",
    "IMLE": "imle",
    "SITD": "seeingdark",
    "SAN": "san",
    "DeepFake": "deepfake",
    "Whichfaceisreal": "whichface",
}



In [ ]:
!git clone https://github.com/PeterWang512/CNNDetection.git

Cloning into 'CNNDetection'...
remote: Enumerating objects: 632, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 632 (delta 25), reused 25 (delta 25), pack-reused 604
Receiving objects: 100% (632/632), 6.36 MiB | 18.83 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
%cd /content/CNNDetection

/content/CNNDetection


# Download the test dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/EECS6322/CNN_synth_testset.zip ./test.zip

In [ ]:
!mkdir ./test_dir/
!unzip ./test.zip -d ./test_dir
!rm ./test

mkdir: cannot create directory ‘./test’: File exists
Archive:  ./test
checkdir:  cannot create extraction directory: ./test
           File exists


In [ ]:
# models to evaluate
models_dir = "~/content/models/models"

# data to use for evaluation
eval_dir = "~/content/CNNDetection/dataset/test"


In [ ]:
import torch
from gandetect.dataloader import load_data
from gandetect.transforms import TEST_TRANSFORM, TEST_TRANSFORM_DCT
import numpy as np
import multiprocessing as mp
from PIL import Image
from sklearn.metrics import average_precision_score
from tqdm import tqdm

EVAL_DATASETS = {
    "ProGAN": "progan"
}

batch_size = 64

def evaluate_model(model_path, dataset_path):
    # prepare the dataset for testing
    transform = TEST_TRANSFORM
    dataloader = torch.utils.data.DataLoader(
                  torch.utils.data.ConcatDataset(
                    load_data(dataset_path, transformations=transform)),
                  num_workers=mp.cpu_count(), batch_size=batch_size)


    model = torch.load(model_path)
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    mAP = []

    with torch.no_grad():
        print(f"Evaluating model...{model_path}")
        y_true = []
        y_pred = []
        for batch in tqdm(dataloader):
            ims, labels = batch
            ims = ims.to(device)
            preds = model(ims).sigmoid().float().flatten().tolist()

            y_true.extend(labels.float().flatten().tolist())
            y_pred.extend(preds)
            
        print(average_precision_score(y_true, y_pred))
        mAP.append(average_precision_score(y_true, y_pred))

In [ ]:
model_path = "/content/models/models/16_datasets/blur_jpeg_5/resnet50/checkpoint.pth"
dataset_path = "/content/test_dir"

evaluate_model(model_path, dataset_path)

In [ ]:
import argparse
import multiprocessing as mp
from pathlib import Path

import numpy as np
import torch
from PIL import Image
from sklearn.metrics import average_precision_score
from tqdm import tqdm

from gandetect.dataloader import load_data
from gandetect.transforms import TEST_TRANSFORM, TEST_TRANSFORM_DCT
from gandetect.utils import set_seed

EVAL_DATASETS = {
    "ProGAN": "progan",
    "StyleGAN": "stylegan",
    "StyleGAN2": "stylegan2",
    "BigGAN": "biggan",
    "CycleGAN": "cyclegan",
    "StarGAN": "stargan",
    "GauGAN": "gaugan",
    "CRN": "crn",
    "IMLE": "imle",
    "SITD": "seeingdark",
    "SAN": "san",
    "DeepFake": "deepfake",
    "Whichfaceisreal": "whichface",
}


def main(args):
    set_seed(42)
    models_paths = [str(path.absolute())
                    for path in Path(args.MODELS).rglob("checkpoint.pth")]

    # prep datasets
    datasets = {name: f"{args.DIR}/{path}"
                for name, path in EVAL_DATASETS.items()}

    if not args.include_new:
        del datasets["StyleGAN2"]
        del datasets["Whichfaceisreal"]

    transfrom = TEST_TRANSFORM_DCT if args.dct else TEST_TRANSFORM
    datasets = {name: torch.utils.data.DataLoader(
        torch.utils.data.ConcatDataset(
            load_data(dataset_path, transformations=transfrom)),
        num_workers=mp.cpu_count(), batch_size=args.batch_size) for name, dataset_path in datasets.items()}

    with open(f"{args.result_file_name}.csv", "w+") as csv, open(f"{args.result_file_name}.tex", "w+") as tex:
        # headers
        csv.write(f"Name,{','.join(datasets.keys())},mAP\n")

        tex.write(r"\begin{tabular}{" + f"{'cc' + len(datasets)* 'c'}"+"}\n")
        tex.write("\\toprule\n")
        tex.write(r"Name &")
        for name in datasets.keys():
            tex.write(f"{name} &")
        tex.write(r"mAP \\" + "\n")

        for model_path in models_paths:
            print(f"Evaluating {model_path}")
            model = torch.load(model_path)
            model = model.eval()

            csv.write(f"{model_path},")

            # name needs to be changed before table compiles
            tex.write(f"{model_path} &")

            device = torch.device(
                "cuda") if torch.cuda.is_available() else torch.device("cpu")

            mAP = []
            print(f"============================")
            with torch.no_grad():
                for name, dataloader in datasets.items():
                    print(f"Evaluating {name}!")
                    y_true, y_pred = [], []
                    for batch in tqdm(dataloader):
                        imgs, labels = batch
                        imgs = imgs.to(device)
                        predictions = model(
                            imgs).sigmoid().float().flatten().tolist()

                        y_true.extend(labels.float().flatten().tolist())
                        y_pred.extend(predictions)

                    ap = average_precision_score(y_true=y_true, y_score=y_pred)
                    print(f"{name} - {ap:.2%}")
                    csv.write(f"{ap * 100:.1f},")
                    tex.write(f"{ap * 100:.1f} &")
                    mAP.append(ap)
                    print(f"============================")

            mAP = np.mean(mAP)
            print(f"mAP: {mAP:.2%}")
            csv.write(f"{mAP * 100:.1f}\n")
            tex.write(f"{mAP * 100:.1f}" + r"\\ \midrule" + "\n")

        tex.write(r"\end{tabular}")


def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument("MODELS", help="Model to use.", type=str)
    parser.add_argument("DIR", help="Directory to analyse.", type=str)

    default_batch = 64
    parser.add_argument(
        "--batch_size", help=f"Batch size to use default: {default_batch}.", default=default_batch)

    parser.add_argument(
        "--include_new", help="Includes StyleGAN2 and Whichfaceisreal in the evaluation.", action="store_true")
    parser.add_argument(
        "--dct", help="Use DCT preprocessing instead.", action="store_true")

    results_file = "results"
    parser.add_argument(
        "--result_file_name", "-r", help="Result file name for csv/tex; default: {results_file}.", type=str, default=results_file)

    return parser.parse_args()


In [ ]:

# models to evaluate
models_dir = "~/content/models/models"

# data to use for evaluation
eval_dir = "~/content/CNNDetection/dataset/test"

In [ ]:
%cd ./ReproducabilityCNNEasyToSpot/

/content/ReproducabilityCNNEasyToSpot


In [ ]:
!ls

CNNDetection  gandetect  models.zip		       results.csv  sample_data
drive	      models	 ReproducabilityCNNEasyToSpot  results.tex


In [ ]:
import gandetect

In [ ]:
!pwd

/content


In [ ]:
%cd ./ReproducabilityCNNEasyToSpot

/content/ReproducabilityCNNEasyToSpot


In [ ]:
!python evaluate.py ~/content/models/models/20_datasets/blur_jpeg_1/  ~/content/test_dir/